# Import Required Libraries

In [5]:
import pandas as pd
import os
import shutil
import zipfile
from datetime import datetime
from pathlib import Path
import re

# Load and Explore the Dataset

In [29]:
csv_path = r"../data/metadata/streetcare-drift-dataset-2021-2025.csv"

df = pd.read_csv(csv_path.replace('\\', '/'), low_memory=False)

print(f"Dataset shape: {df.shape}")
print(f"\nColumn names:")
print(df.columns.tolist())
print(f"\nFirst few rows:")
print(df.head())

Dataset shape: (588011, 16)

Column names:
['id', 'serial', 'date', 'hostname', 'lat', 'lon', 'image_name', 'time_tag', 'fault_detected', 'confidence', 'daynight', 'red', 'green', 'blue', 'relative_centroid_drift', 'relative_recon_error']

First few rows:
       id    serial                 date           hostname        lat  \
0  6964.0  RSE-19-C  2021-05-18 14:02:46  umbrella-258de557  51.505252   
1  6994.0  RSE-19-C  2021-05-18 15:08:15  umbrella-258de557  51.505305   
2  7000.0  RSE-19-C  2021-05-18 16:02:02  umbrella-258de557  51.505307   
3  7022.0  RSE-19-C  2021-05-18 17:02:35  umbrella-258de557  51.505275   
4  7050.0  RSE-19-C  2021-05-18 18:06:49  umbrella-258de557  51.505298   

        lon                            image_name time_tag  fault_detected  \
0 -2.492399  d856ecdab091aad040df0f863bb21f4b.jpg  2021-Q2             0.0   
1 -2.492470  6507ad5917aef93b8d52610c28944d0f.jpg  2021-Q2             0.0   
2 -2.492412  dbc4d613c97502a59e2bb14a6c8296fd.jpg  2021-Q2       

# Parse Quarter from time_tag Column

In [30]:
print(f"\n{'='*80}")
print(f"PARSING QUARTER FROM time_tag COLUMN")
print(f"{'='*80}")

def extract_quarter_from_time_tag(time_tag):
    """
    Extract quarter from time_tag string like '2021-Q1', '2022-Q3', etc.
    
    Args:
        time_tag: String like '2021-Q1' or '2021-Q3'
    
    Returns:
        int: Quarter number (1 or 3), or None
    """
    if pd.isna(time_tag):
        return None
    
    time_tag = str(time_tag).upper().strip()
    
    if 'Q1' in time_tag:
        return 1
    elif 'Q3' in time_tag:
        return 3
    
    return None

# Apply quarter extraction
df['quarter'] = df['time_tag'].apply(extract_quarter_from_time_tag)

print(f"\nQuarter extraction from time_tag:")
print(f"Q1 records: {(df['quarter'] == 1).sum()}")
print(f"Q3 records: {(df['quarter'] == 3).sum()}")
print(f"Other/None: {df['quarter'].isna().sum()}")

print(f"\nSample time_tag and extracted quarter:")
sample_df = df[['time_tag', 'serial', 'quarter']].head(10)
print(sample_df.to_string(index=False))


PARSING QUARTER FROM time_tag COLUMN

Quarter extraction from time_tag:
Q1 records: 125532
Q3 records: 125285
Other/None: 337194

Sample time_tag and extracted quarter:
time_tag   serial  quarter
 2021-Q2 RSE-19-C      NaN
 2021-Q2 RSE-19-C      NaN
 2021-Q2 RSE-19-C      NaN
 2021-Q2 RSE-19-C      NaN
 2021-Q2 RSE-19-C      NaN
 2021-Q2 RSE-19-C      NaN
 2021-Q2 RSE-19-C      NaN
 2021-Q2 RSE-19-C      NaN
 2021-Q2 RSE-19-C      NaN
 2021-Q2 RSE-19-C      NaN


# Filter for Daytime Using daynight Column

In [31]:
print(f"\n{'='*80}")
print(f"DAYTIME FILTERING USING daynight COLUMN")
print(f"{'='*80}")

# Filter for Q1 and Q3 quarters first
df_q1q3 = df[df['quarter'].isin([1, 3])].copy()

# Filter for daytime (daynight == 0.0 means daytime)
df_q1q3_daytime = df_q1q3[df_q1q3['daynight'] == 0.0].copy()

print(f"Total records: {len(df)}")
print(f"Q1/Q3 records: {len(df_q1q3)}")
print(f"Q1/Q3 daytime records (daynight=0): {len(df_q1q3_daytime)}")

print(f"\nDaytime distribution by quarter:")
print(f"Q1 daytime records: {(df_q1q3_daytime['quarter'] == 1).sum()}")
print(f"Q3 daytime records: {(df_q1q3_daytime['quarter'] == 3).sum()}")

print(f"\nDaynight value distribution (all Q1/Q3):")
print(df_q1q3['daynight'].value_counts())


DAYTIME FILTERING USING daynight COLUMN
Total records: 588011
Q1/Q3 records: 250817
Q1/Q3 daytime records (daynight=0): 128483

Daytime distribution by quarter:
Q1 daytime records: 52543
Q3 daytime records: 75940

Daynight value distribution (all Q1/Q3):
daynight
0.0    128483
1.0    122334
Name: count, dtype: int64


# Temporal Breakdown Analysis

In [32]:
print(f"\n{'='*80}")
print(f"TEMPORAL BREAKDOWN")
print(f"{'='*80}")

# Parse date column for analysis
df_q1q3_daytime['date'] = pd.to_datetime(df_q1q3_daytime['date'], errors='coerce')
df_q1q3_daytime['year'] = df_q1q3_daytime['date'].dt.year
df_q1q3_daytime['month'] = df_q1q3_daytime['date'].dt.month

print(f"\nYear distribution (daytime Q1/Q3):")
print(df_q1q3_daytime['year'].value_counts().sort_index())

print(f"\nQuarter-Year breakdown:")
qy_breakdown = df_q1q3_daytime.groupby(['year', 'quarter']).size().reset_index(name='count')
print(qy_breakdown.to_string(index=False))

print(f"\nMonth distribution by quarter:")
m_breakdown = df_q1q3_daytime.groupby(['quarter', 'month']).size().reset_index(name='count')
print(m_breakdown.to_string(index=False))


TEMPORAL BREAKDOWN

Year distribution (daytime Q1/Q3):
year
2021    22981
2022    28728
2023    42364
2024    27755
2025     6655
Name: count, dtype: int64

Quarter-Year breakdown:
 year  quarter  count
 2021      3.0  22981
 2022      1.0  17907
 2022      3.0  10821
 2023      1.0  18794
 2023      3.0  23570
 2024      1.0  11267
 2024      3.0  16488
 2025      1.0   4575
 2025      3.0   2080

Month distribution by quarter:
 quarter  month  count
     1.0      1  16127
     1.0      2  18639
     1.0      3  16840
     1.0      4      6
     1.0      5     42
     1.0      6    224
     1.0      7    134
     1.0      8    118
     1.0      9    126
     1.0     10    259
     1.0     11      1
     1.0     12     27
     3.0      1    101
     3.0      2    321
     3.0      3    202
     3.0      4    636
     3.0      5    661
     3.0      6    604
     3.0      7  30109
     3.0      8  18054
     3.0      9  23982
     3.0     10    433
     3.0     11    698
     3.0     1

In [33]:
# ============================================================================
# STEP 1: FILTER AND PREPARE METADATA (NO IMAGE EXTRACTION YET)
# ============================================================================

print(f"\n{'='*80}")
print(f"STEP 1: FILTER METADATA WITH ALL REQUIREMENTS")
print(f"{'='*80}")

# Extract quarter from time_tag
df_q1q3_daytime['_quarter'] = df_q1q3_daytime['time_tag'].apply(extract_quarter_from_time_tag)

# Filter: Q1/Q3 AND daytime only (daynight == 0.0)
df_filtered_metadata = df_q1q3_daytime[
    (df_q1q3_daytime['daynight'] == 0.0) & 
    (df_q1q3_daytime['_quarter'].isin([1, 3]))
].copy()

# Remove temporary column
df_filtered_metadata = df_filtered_metadata.drop(columns=['_quarter'])

print(f"Total filtered metadata records: {len(df_filtered_metadata)}")
print(f"Q1 records: {len(df_filtered_metadata[df_filtered_metadata['time_tag'].str.contains('Q1', na=False)])}")
print(f"Q3 records: {len(df_filtered_metadata[df_filtered_metadata['time_tag'].str.contains('Q3', na=False)])}")

# Verify all records are daytime
all_daytime = (df_filtered_metadata['daynight'] == 0.0).all()
print(f"Daytime verification: {'✅' if all_daytime else '⚠️'} (All daynight == 0.0)")

# Sample if needed (keep up to 2000 per quarter)
sample_size = 2000
q1_filtered = df_filtered_metadata[df_filtered_metadata['time_tag'].str.contains('Q1', na=False)].copy()
q3_filtered = df_filtered_metadata[df_filtered_metadata['time_tag'].str.contains('Q3', na=False)].copy()

q1_data = q1_filtered.sample(n=min(sample_size, len(q1_filtered)), random_state=42) if len(q1_filtered) > 0 else q1_filtered
q3_data = q3_filtered.sample(n=min(sample_size, len(q3_filtered)), random_state=42) if len(q3_filtered) > 0 else q3_filtered

df_sampled = pd.concat([q1_data, q3_data]).reset_index(drop=True)

print(f"\nAfter sampling (up to {sample_size} per quarter):")
print(f"Q1 samples: {len(q1_data)}")
print(f"Q3 samples: {len(q3_data)}")
print(f"Total samples for extraction: {len(df_sampled)}")

# Reset index for proper tracking
q1_data = q1_data.reset_index(drop=True)
q3_data = q3_data.reset_index(drop=True)


STEP 1: FILTER METADATA WITH ALL REQUIREMENTS
Total filtered metadata records: 128483
Q1 records: 52543
Q3 records: 75940
Daytime verification: ✅ (All daynight == 0.0)

After sampling (up to 2000 per quarter):
Q1 samples: 2000
Q3 samples: 2000
Total samples for extraction: 4000


# Save Prepared Metadata Before Extraction

In [34]:
print(f"\n{'='*80}")
print(f"STEP 2: SAVE PREPARED METADATA (BEFORE IMAGE EXTRACTION)")
print(f"{'='*80}")

# Save the sampled metadata that will be used for image extraction
metadata_output = "../data/metadata/q1q3_daytime_extracted.csv"
os.makedirs(os.path.dirname(metadata_output), exist_ok=True)
df_sampled.to_csv(metadata_output, index=False)

print(f"\nPrepared metadata saved: {metadata_output}")
print(f"   Total records: {len(df_sampled)}")
print(f"   Q1 records: {len(q1_data)}")
print(f"   Q3 records: {len(q3_data)}")
print(f"   Columns: {list(df_sampled.columns)}")

# Create a set of image_names from metadata for validation
metadata_image_names = set(df_sampled['image_name'].values)
print(f"\nImage names in metadata: {len(metadata_image_names)}")


STEP 2: SAVE PREPARED METADATA (BEFORE IMAGE EXTRACTION)

Prepared metadata saved: ../data/metadata/q1q3_daytime_extracted.csv
   Total records: 4000
   Q1 records: 2000
   Q3 records: 2000
   Columns: ['id', 'serial', 'date', 'hostname', 'lat', 'lon', 'image_name', 'time_tag', 'fault_detected', 'confidence', 'daynight', 'red', 'green', 'blue', 'relative_centroid_drift', 'relative_recon_error', 'quarter', 'year', 'month']

Image names in metadata: 4000


In [35]:
print(f"\n{'='*80}")
print(f"SAMPLING BALANCED DATASETS")
print(f"{'='*80}")

# Extract Q1 and Q3
q1_full = df_q1q3_daytime[df_q1q3_daytime['quarter'] == 1]
q3_full = df_q1q3_daytime[df_q1q3_daytime['quarter'] == 3]

# Sample balanced datasets (up to 2000 each)
sample_size = 2000
q1_data = q1_full.sample(n=min(sample_size, len(q1_full)), random_state=42)
q3_data = q3_full.sample(n=min(sample_size, len(q3_full)), random_state=42)

df_sampled = pd.concat([q1_data, q3_data]).reset_index(drop=True)

print(f"Q1 samples: {len(q1_data)}")
print(f"Q3 samples: {len(q3_data)}")
print(f"Total Q1/Q3 samples: {len(df_sampled)}")

print(f"\nQ1 breakdown by year:")
print(q1_data['year'].value_counts().sort_index())

print(f"\nQ3 breakdown by year:")
print(q3_data['year'].value_counts().sort_index())

print(f"\nDaynight verification (should all be 0.0 for daytime):")
print(df_sampled['daynight'].value_counts())


SAMPLING BALANCED DATASETS
Q1 samples: 2000
Q3 samples: 2000
Total Q1/Q3 samples: 4000

Q1 breakdown by year:
year
2022    665
2023    725
2024    422
2025    188
Name: count, dtype: int64

Q3 breakdown by year:
year
2021    589
2022    284
2023    637
2024    442
2025     48
Name: count, dtype: int64

Daynight verification (should all be 0.0 for daytime):
daynight
0.0    4000
Name: count, dtype: int64


# Create Output Folders

In [36]:
output_base = "../data/organized_images"
q1_folder = os.path.join(output_base, "Q1")
q3_folder = os.path.join(output_base, "Q3")

os.makedirs(q1_folder, exist_ok=True)
os.makedirs(q3_folder, exist_ok=True)

print(f"\n{'='*80}")
print(f"OUTPUT FOLDER CREATION")
print(f"{'='*80}")
print(f"Q1 folder: {os.path.abspath(q1_folder)}")
print(f"Q3 folder: {os.path.abspath(q3_folder)}")


OUTPUT FOLDER CREATION
Q1 folder: c:\Users\melko\Capstone\data\organized_images\Q1
Q3 folder: c:\Users\melko\Capstone\data\organized_images\Q3


# Define ZIP Extraction Function

In [37]:
def extract_image_from_zip(serial, image_name, raw_zip_folder='../data/raw'):
    """
    Extract image from ZIP file based on serial (camera identifier).
    
    Args:
        serial: Serial/camera identifier from metadata (e.g., 'RSE-6-C')
        image_name: Image filename to extract
        raw_zip_folder: Folder containing ZIP files
    
    Returns:
        bytes: Image data if found, None otherwise
    """
    if pd.isna(serial) or pd.isna(image_name):
        return None
    
    serial = str(serial).strip()
    image_name = str(image_name).strip()
    
    # Construct possible ZIP file paths based on serial
    zip_paths = [
        os.path.join(raw_zip_folder, f"{serial}.zip"),
        os.path.join(raw_zip_folder, serial, f"{serial}.zip"),
        os.path.join(raw_zip_folder, f"{serial}.ZIP"),
        os.path.join(raw_zip_folder, f"{serial.lower()}.zip"),
    ]
    
    for zip_path in zip_paths:
        if os.path.exists(zip_path):
            try:
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    file_list = zip_ref.namelist()
                    
                    # Try exact match first
                    if image_name in file_list:
                        return zip_ref.read(image_name)
                    
                    # Try matching filename only (in case of nested folders)
                    for file in file_list:
                        if file.endswith(image_name) or file.endswith(os.path.basename(image_name)):
                            return zip_ref.read(file)
                    
            except zipfile.BadZipFile:
                pass
            except Exception as e:
                pass
    
    return None

print("ZIP extraction function defined")

ZIP extraction function defined


# Extract Q1 Images from ZIP Files

In [38]:
print(f"\n{'='*80}")
print(f"STEP 3: EXTRACT Q1 IMAGES (MATCHING METADATA)")
print(f"{'='*80}")

q1_count = 0
q1_missing = 0
q1_errors = 0

print(f"\nProcessing Q1 daytime images from metadata ({len(q1_data)} records)...")

for idx, row in q1_data.iterrows():
    image_name = row['image_name']
    serial = row.get('serial', None)
    
    # Extract image from ZIP using serial
    image_data = extract_image_from_zip(serial, image_name)
    
    if image_data is None:
        q1_missing += 1
        print(f"Q1[{idx}]: Missing - {image_name}")
        continue
    
    # Save to Q1 folder
    dest_path = os.path.join(q1_folder, os.path.basename(image_name))
    try:
        os.makedirs(os.path.dirname(dest_path), exist_ok=True)
        with open(dest_path, 'wb') as f:
            f.write(image_data)
        q1_count += 1
        
        if (q1_count) % 250 == 0:
            print(f"✓ Q1: {q1_count}/{len(q1_data)} images extracted")
    except Exception as e:
        q1_errors += 1
        print(f"Q1[{idx}]: Error - {str(e)}")

print(f"\n Q1 extraction complete:")
print(f"   Extracted: {q1_count}/{len(q1_data)}")
print(f"   Missing: {q1_missing}")
print(f"   Errors: {q1_errors}")


STEP 3: EXTRACT Q1 IMAGES (MATCHING METADATA)

Processing Q1 daytime images from metadata (2000 records)...
Q1[100279]: Missing - a2c06f2f65290078edd967604927132e.jpg
Q1[96864]: Missing - 146daa6207571ad0605ae4d816bc2ed6.jpg
Q1[96515]: Missing - 6b7168f13e48f57ae7544feac6fa4c5c.jpg
Q1[103621]: Missing - e8f107a0ad2c894a8be968d47d7a33fb.jpg
Q1[103481]: Missing - c857920dc56445208feb3def9cebf2f8.jpg
Q1[100143]: Missing - 7189866352e82d2f6d1329c86575fdde.jpg
Q1[96143]: Missing - f0ecd06ea7f5373474871f3dc8923b03.jpg
Q1[103714]: Missing - 8c9e7e95198f0452ba2fb82a54fa7e6b.jpg
Q1[99932]: Missing - 59d95dea0eb0270927678a9c1b4e452e.jpg
Q1[96436]: Missing - 722666ffaff0916824b043bc411aca5a.jpg
Q1[96070]: Missing - 4f9617b315bbe7a2ec728579b10b1f1a.jpg
   ✓ Q1: 250/2000 images extracted
Q1[103654]: Missing - 88de2d45d794d973a1c33d23250db227.jpg
Q1[99688]: Missing - 0a133f090f155160e576a51cf4868086.jpg
Q1[103524]: Missing - e3aeacd2b9f52b2196c04e3083d257fd.jpg
Q1[100062]: Missing - ad2b3dada44a1a1

# Extract Q3 Images from ZIP Files

In [39]:
print(f"\n{'='*80}")
print(f"STEP 4: EXTRACT Q3 IMAGES (MATCHING METADATA)")
print(f"{'='*80}")

q3_count = 0
q3_missing = 0
q3_errors = 0

print(f"\nProcessing Q3 daytime images from metadata ({len(q3_data)} records)...")

for idx, row in q3_data.iterrows():
    image_name = row['image_name']
    serial = row.get('serial', None)
    
    # Extract image from ZIP using serial
    image_data = extract_image_from_zip(serial, image_name)
    
    if image_data is None:
        q3_missing += 1
        print(f"Q3[{idx}]: Missing - {image_name}")
        continue
    
    # Save to Q3 folder
    dest_path = os.path.join(q3_folder, os.path.basename(image_name))
    try:
        os.makedirs(os.path.dirname(dest_path), exist_ok=True)
        with open(dest_path, 'wb') as f:
            f.write(image_data)
        q3_count += 1
        
        if (q3_count) % 250 == 0:
            print(f"   ✓ Q3: {q3_count}/{len(q3_data)} images extracted")
    except Exception as e:
        q3_errors += 1
        print(f"Q3[{idx}]: Error - {str(e)}")

print(f"\n Q3 extraction complete:")
print(f"   Extracted: {q3_count}/{len(q3_data)}")
print(f"   Missing: {q3_missing}")
print(f"   Errors: {q3_errors}")


STEP 4: EXTRACT Q3 IMAGES (MATCHING METADATA)

Processing Q3 daytime images from metadata (2000 records)...
Q3[102292]: Missing - 0446f1ff5c3364fe3024cf990fd63c99.jpg
Q3[94693]: Missing - 48d7e70b53ecc3421c021aba9b1a702d.jpg
Q3[104363]: Missing - 28ef27463024022745e0304feb042ebc.jpg
Q3[94611]: Missing - 01b5fe0beea19d0aea048c25f9ea5b95.jpg
Q3[101844]: Missing - 3fdd0f6de054a9f100b5aa52f1c811ca.jpg
Q3[94427]: Missing - 9417f4a6037cde07ab1931d4adb0e1ee.jpg
Q3[104490]: Missing - b0c57a93d31b341d5415dd8054970957.jpg
Q3[104135]: Missing - 4b68ed28e29746c05fe7a50fa2393a97.jpg
Q3[104518]: Missing - 3ff4c1113baa53211554f308e5b158c0.jpg
Q3[101850]: Missing - 49d135aa7cac986f85fa819bf0656095.jpg
Q3[95208]: Missing - 9352bcf0a356a1499c44fa8a39576ba9.jpg
Q3[104095]: Missing - 37662aca4d131962e00b88559b4c9d52.jpg
Q3[98313]: Missing - d39beba6fcbf54228ce74f79cba6b0db.jpg
Q3[94700]: Missing - d49b00703c4466b020e47bf52c0a4521.jpg
   ✓ Q3: 250/2000 images extracted
Q3[102250]: Missing - 55a3482af449a1

# Extraction Summary Report

In [23]:
print(f"\n{'='*80}")
print(f"EXTRACTION SUMMARY REPORT")
print(f"{'='*80}")

total_missing = q1_missing + q3_missing
total_errors = q1_errors + q3_errors
total_extracted = q1_count + q3_count
total_samples = len(df_sampled)

print(f"\n Q1 Results:")
print(f"   Extracted: {q1_count}/{len(q1_data)} ({100*q1_count/len(q1_data):.1f}%)")
print(f"   Not found: {q1_missing}")
print(f"   Errors: {q1_errors}")

print(f"\n Q3 Results:")
print(f"   Extracted: {q3_count}/{len(q3_data)} ({100*q3_count/len(q3_data):.1f}%)")
print(f"   Not found: {q3_missing}")
print(f"   Errors: {q3_errors}")

print(f"\n Overall Statistics:")
print(f"   Total extracted: {total_extracted}/{total_samples} ({100*total_extracted/total_samples:.1f}%)")
print(f"   Total not found: {total_missing}")
print(f"   Total errors: {total_errors}")


EXTRACTION SUMMARY REPORT

 Q1 Results:
   Extracted: 1919/2000 (96.0%)
   Not found: 81
   Errors: 0

 Q3 Results:
   Extracted: 1889/2000 (94.5%)
   Not found: 111
   Errors: 0

 Overall Statistics:
   Total extracted: 3808/4000 (95.2%)
   Total not found: 192
   Total errors: 0


# Filter Metadata to Remove Missing Images

In [24]:
print(f"\n{'='*80}")
print(f"STEP 4.5: FILTER METADATA TO MATCH EXTRACTED IMAGES ONLY")
print(f"{'='*80}")

# Get all successfully extracted image basenames
q1_extracted_files = set([f for f in os.listdir(q1_folder) if os.path.isfile(os.path.join(q1_folder, f))]) if os.path.exists(q1_folder) else set()
q3_extracted_files = set([f for f in os.listdir(q3_folder) if os.path.isfile(os.path.join(q3_folder, f))]) if os.path.exists(q3_folder) else set()

all_extracted_files = q1_extracted_files | q3_extracted_files

print(f"Q1 extracted files: {len(q1_extracted_files)}")
print(f"Q3 extracted files: {len(q3_extracted_files)}")
print(f"Total extracted files: {len(all_extracted_files)}")

# Filter metadata to keep only records with extracted images
print(f"\nBefore filtering metadata:")
print(f"  Total metadata records: {len(df_sampled)}")
print(f"  Q1 records: {len(q1_data)}")
print(f"  Q3 records: {len(q3_data)}")

# Create boolean mask for records with extracted images
df_sampled['_has_image'] = df_sampled['image_name'].apply(lambda x: os.path.basename(x) in all_extracted_files)
q1_data['_has_image'] = q1_data['image_name'].apply(lambda x: os.path.basename(x) in all_extracted_files)
q3_data['_has_image'] = q3_data['image_name'].apply(lambda x: os.path.basename(x) in all_extracted_files)

# Count missing records
q1_missing_records = (~q1_data['_has_image']).sum()
q3_missing_records = (~q3_data['_has_image']).sum()
total_missing_records = (~df_sampled['_has_image']).sum()

print(f"\nRecords without extracted images:")
print(f"  Q1: {q1_missing_records}")
print(f"  Q3: {q3_missing_records}")
print(f"  Total: {total_missing_records}")

# Filter to keep only records with images
df_sampled_filtered = df_sampled[df_sampled['_has_image']].copy()
q1_data_filtered = q1_data[q1_data['_has_image']].copy()
q3_data_filtered = q3_data[q3_data['_has_image']].copy()

# Remove temporary column
df_sampled_filtered = df_sampled_filtered.drop(columns=['_has_image'])
q1_data_filtered = q1_data_filtered.drop(columns=['_has_image'])
q3_data_filtered = q3_data_filtered.drop(columns=['_has_image'])

# Also clean up original dataframes
df_sampled = df_sampled.drop(columns=['_has_image'])
q1_data = q1_data.drop(columns=['_has_image'])
q3_data = q3_data.drop(columns=['_has_image'])

print(f"\nAfter filtering metadata (keeping only extracted images):")
print(f"  Total metadata records: {len(df_sampled_filtered)}")
print(f"  Q1 records: {len(q1_data_filtered)}")
print(f"  Q3 records: {len(q3_data_filtered)}")

# Update df_sampled to use the filtered version
df_sampled = df_sampled_filtered
q1_data = q1_data_filtered
q3_data = q3_data_filtered

# Resave metadata with only matched records
print(f"\nRe-saving metadata with only extracted image records...")
metadata_output = "../data/metadata/q1q3_daytime_extracted.csv"
os.makedirs(os.path.dirname(metadata_output), exist_ok=True)
df_sampled.to_csv(metadata_output, index=False)

print(f"Updated metadata saved: {metadata_output}")
print(f"   Records: {len(df_sampled)}")
print(f"   Q1: {len(q1_data)}")
print(f"   Q3: {len(q3_data)}")


STEP 4.5: FILTER METADATA TO MATCH EXTRACTED IMAGES ONLY
Q1 extracted files: 1919
Q3 extracted files: 1889
Total extracted files: 3808

Before filtering metadata:
  Total metadata records: 4000
  Q1 records: 2000
  Q3 records: 2000

Records without extracted images:
  Q1: 81
  Q3: 111
  Total: 192

After filtering metadata (keeping only extracted images):
  Total metadata records: 3808
  Q1 records: 1919
  Q3 records: 1889

Re-saving metadata with only extracted image records...
Updated metadata saved: data/metadata/q1q3_daytime_extracted.csv
   Records: 3808
   Q1: 1919
   Q3: 1889


# Verify Organization Results

In [25]:
print(f"\n{'='*80}")
print(f"VERIFICATION RESULTS")
print(f"{'='*80}\n")

# Check Q1 folder
q1_files = [f for f in os.listdir(q1_folder) if os.path.isfile(os.path.join(q1_folder, f))] if os.path.exists(q1_folder) else []
q1_size = sum(os.path.getsize(os.path.join(q1_folder, f)) for f in q1_files)

print(f"Q1 Folder Statistics:")
print(f"  Total files: {len(q1_files)}")
print(f"  Total size: {q1_size / (1024**2):.2f} MB")
if len(q1_files) > 0:
    print(f"  Sample files: {q1_files[:3]}")
    print(f"  Avg file size: {q1_size / len(q1_files) / 1024:.2f} KB")

# Check Q3 folder
q3_files = [f for f in os.listdir(q3_folder) if os.path.isfile(os.path.join(q3_folder, f))] if os.path.exists(q3_folder) else []
q3_size = sum(os.path.getsize(os.path.join(q3_folder, f)) for f in q3_files)

print(f"\nQ3 Folder Statistics:")
print(f"  Total files: {len(q3_files)}")
print(f"  Total size: {q3_size / (1024**2):.2f} MB")
if len(q3_files) > 0:
    print(f"  Sample files: {q3_files[:3]}")
    print(f"  Avg file size: {q3_size / len(q3_files) / 1024:.2f} KB")

# Summary statistics
total_organized = len(q1_files) + len(q3_files)
total_disk_space = (q1_size + q3_size) / (1024**2)

print(f"\n{'='*80}")
print(f"ORGANIZATION SUMMARY")
print(f"{'='*80}")
print(f"Total images organized: {total_organized}")
print(f"Q1 images: {len(q1_files)} ({100*len(q1_files)/max(total_organized,1):.1f}%)")
print(f"Q3 images: {len(q3_files)} ({100*len(q3_files)/max(total_organized,1):.1f}%)")
print(f"Total disk space used: {total_disk_space:.2f} MB")


VERIFICATION RESULTS

Q1 Folder Statistics:
  Total files: 1919
  Total size: 892.64 MB
  Sample files: ['00044a0a3a5519829765da599eade01d.jpg', '00082df0fd2b48485ad9d85362d64faf.jpg', '0057ae8682c5bd2a3fa91fdadbcfb582.jpg']
  Avg file size: 476.32 KB

Q3 Folder Statistics:
  Total files: 1889
  Total size: 865.97 MB
  Sample files: ['0027551377f3f1a5a80361c791cbc096.jpg', '006bd7ece6a8ab701c3401b301a58882.jpg', '00afa64cadb9325c4bb713fe87cd3a4f.jpg']
  Avg file size: 469.43 KB

ORGANIZATION SUMMARY
Total images organized: 3808
Q1 images: 1919 (50.4%)
Q3 images: 1889 (49.6%)
Total disk space used: 1758.61 MB


# Save Extracted Metadata

In [26]:
print(f"\n{'='*80}")
print(f"STEP 5: VERIFY METADATA AND IMAGES COUNT MATCH")
print(f"{'='*80}")

# Load the current metadata (already filtered to only extracted images)
df_q1q3_extracted = pd.read_csv(metadata_output)

# Get actual extracted files
q1_files_actual = sorted([f for f in os.listdir(q1_folder) if os.path.isfile(os.path.join(q1_folder, f))]) if os.path.exists(q1_folder) else []
q3_files_actual = sorted([f for f in os.listdir(q3_folder) if os.path.isfile(os.path.join(q3_folder, f))]) if os.path.exists(q3_folder) else []

# Get metadata counts
q1_metadata_count = len(df_q1q3_extracted[df_q1q3_extracted['time_tag'].str.contains('Q1', na=False)])
q3_metadata_count = len(df_q1q3_extracted[df_q1q3_extracted['time_tag'].str.contains('Q3', na=False)])
total_metadata_count = len(df_q1q3_extracted)

# Get actual file counts
q1_files_count = len(q1_files_actual)
q3_files_count = len(q3_files_actual)
total_files_count = q1_files_count + q3_files_count

print(f"\nMETADATA RECORDS:")
print(f"   Q1: {q1_metadata_count}")
print(f"   Q3: {q3_metadata_count}")
print(f"   Total: {total_metadata_count}")

print(f"\nEXTRACTED FILES:")
print(f"   Q1: {q1_files_count}")
print(f"   Q3: {q3_files_count}")
print(f"   Total: {total_files_count}")

# Verify counts match
q1_match = q1_metadata_count == q1_files_count
q3_match = q3_metadata_count == q3_files_count
total_match = total_metadata_count == total_files_count

print(f"\n{'='*80}")
print(f"COUNT VERIFICATION:")
print(f"{'='*80}")
print(f"Q1 Match: {'PASS' if q1_match else 'FAIL'} ({q1_metadata_count} == {q1_files_count})")
print(f"Q3 Match: {'PASS' if q3_match else 'FAIL'} ({q3_metadata_count} == {q3_files_count})")
print(f"Total Match: {'PASS' if total_match else 'FAIL'} ({total_metadata_count} == {total_files_count})")

# Also verify daytime
daytime_check = (df_q1q3_extracted['daynight'] == 0.0).all()
print(f"Daytime Filter: {'PASS' if daytime_check else 'FAIL'} (All daynight == 0.0)")

if not (q1_match and q3_match and total_match):
    print(f"\nWARNING: Counts do not match!")
    print(f"   Missing from metadata: {total_files_count - total_metadata_count}")
    print(f"   Extra in metadata: {total_metadata_count - total_files_count}")
else:
    print(f"\nAll counts verified - metadata and images are synchronized!")


STEP 5: VERIFY METADATA AND IMAGES COUNT MATCH

METADATA RECORDS:
   Q1: 1919
   Q3: 1889
   Total: 3808

EXTRACTED FILES:
   Q1: 1919
   Q3: 1889
   Total: 3808

COUNT VERIFICATION:
Q1 Match: PASS (1919 == 1919)
Q3 Match: PASS (1889 == 1889)
Total Match: PASS (3808 == 3808)
Daytime Filter: PASS (All daynight == 0.0)

All counts verified - metadata and images are synchronized!


# Final Validation

In [27]:
print(f"\n{'='*80}")
print(f"STEP 6: FINAL VALIDATION - ALL COUNTS MUST MATCH")
print(f"{'='*80}")

# Reload metadata to ensure we have the latest version
df_final = pd.read_csv(metadata_output)

# Get actual file lists
q1_files = sorted([f for f in os.listdir(q1_folder) if os.path.isfile(os.path.join(q1_folder, f))]) if os.path.exists(q1_folder) else []
q3_files = sorted([f for f in os.listdir(q3_folder) if os.path.isfile(os.path.join(q3_folder, f))]) if os.path.exists(q3_folder) else []

# Get metadata counts
q1_metadata = len(df_final[df_final['time_tag'].str.contains('Q1', na=False)])
q3_metadata = len(df_final[df_final['time_tag'].str.contains('Q3', na=False)])
total_metadata = len(df_final)

# Get actual file counts
total_files = len(q1_files) + len(q3_files)

print(f"\nFINAL METADATA:")
print(f"   Q1: {q1_metadata}")
print(f"   Q3: {q3_metadata}")
print(f"   Total: {total_metadata}")

print(f"\nFINAL EXTRACTED IMAGES:")
print(f"   Q1: {len(q1_files)}")
print(f"   Q3: {len(q3_files)}")
print(f"   Total: {total_files}")

print(f"\n{'='*80}")
print(f"FINAL MATCH VERIFICATION:")
print(f"{'='*80}")

# Check matches
q1_match = q1_metadata == len(q1_files)
q3_match = q3_metadata == len(q3_files)
total_match = total_metadata == total_files
daytime_check = (df_final['daynight'] == 0.0).all()

print(f"Q1 Match: {'PASS' if q1_match else 'FAIL'} (Metadata={q1_metadata}, Images={len(q1_files)})")
print(f"Q3 Match: {'PASS' if q3_match else 'FAIL'} (Metadata={q3_metadata}, Images={len(q3_files)})")
print(f"Total Match: {'PASS' if total_match else 'FAIL'} (Metadata={total_metadata}, Images={total_files})")
print(f"Daytime Filter: {'PASS' if daytime_check else 'FAIL'} (All daynight == 0.0)")

print(f"\n{'='*80}")
if q1_match and q3_match and total_match and daytime_check:
    print(f"ALL VALIDATIONS PASSED - 100% MATCH!")
    print(f"{'='*80}")
    print(f"\nMetadata and extracted images are perfectly synchronized!")
    print(f"All records are Q1/Q3 daytime only!")
else:
    print(f"VALIDATION FAILED - COUNTS DO NOT MATCH")
    print(f"{'='*80}")
    if not total_match:
        diff = total_metadata - total_files
        if diff > 0:
            print(f"\nExtra {diff} records in metadata without images!")
        else:
            print(f"\nMissing {-diff} metadata records for extracted images!")

print(f"\nSample metadata records (first 5):")
print(df_final[['serial', 'image_name', 'time_tag', 'daynight']].head(5).to_string(index=False))


STEP 6: FINAL VALIDATION - ALL COUNTS MUST MATCH

FINAL METADATA:
   Q1: 1919
   Q3: 1889
   Total: 3808

FINAL EXTRACTED IMAGES:
   Q1: 1919
   Q3: 1889
   Total: 3808

FINAL MATCH VERIFICATION:
Q1 Match: PASS (Metadata=1919, Images=1919)
Q3 Match: PASS (Metadata=1889, Images=1889)
Total Match: PASS (Metadata=3808, Images=3808)
Daytime Filter: PASS (All daynight == 0.0)

ALL VALIDATIONS PASSED - 100% MATCH!

Metadata and extracted images are perfectly synchronized!
All records are Q1/Q3 daytime only!

Sample metadata records (first 5):
    serial                           image_name time_tag  daynight
RSE-A-10-C fa402d436ec9ae7867f86b8029e678c2.jpg  2024-Q1       0.0
  RSE-44-C 2700b230d1488bd1e2773877b0122fff.jpg  2022-Q1       0.0
   RSE-6-C 4edc3249b44e1b747837a3dfebe91203.jpg  2025-Q1       0.0
  RSE-24-C 6763853f5a7523d8d81a571c93e84d03.jpg  2023-Q1       0.0
RSE-A-15-C e45d94b231b17d651e958be757d97700.jpg  2023-Q1       0.0


# Final Summary

In [28]:
print(f"\n{'='*80}")
print(f"ORGANIZATION COMPLETE!")
print(f"{'='*80}")

print(f"\nOutput Directory Structure:")
print(f"   └── data/")
print(f"       ├── organized_images/")
print(f"       │   ├── Q1/ ({len(q1_files)} images, {q1_size / (1024**2):.2f} MB)")
print(f"       │   └── Q3/ ({len(q3_files)} images, {q3_size / (1024**2):.2f} MB)")
print(f"       └── metadata/")
print(f"           └── q1q3_daytime_extracted.csv ({len(df_q1q3_extracted)} records)")

print(f"\nSummary Statistics:")
print(f"   Total daytime records sampled (Q1/Q3): {len(df_sampled)}")
print(f"   Total extracted images: {total_organized}")
print(f"   Extraction success rate: {100*total_organized/len(df_sampled):.1f}%")
print(f"   Total metadata records: {len(df_q1q3_extracted)} (only extracted)")
print(f"   Total disk space: {total_disk_space:.2f} MB")
print(f"   Processing time: Complete")

print(f"\nDataset ready for feature extraction and drift analysis!")


ORGANIZATION COMPLETE!

Output Directory Structure:
   └── data/
       ├── organized_images/
       │   ├── Q1/ (1919 images, 892.64 MB)
       │   └── Q3/ (1889 images, 865.97 MB)
       └── metadata/
           └── q1q3_daytime_extracted.csv (3808 records)

Summary Statistics:
   Total daytime records sampled (Q1/Q3): 3808
   Total extracted images: 3808
   Extraction success rate: 100.0%
   Total metadata records: 3808 (only extracted)
   Total disk space: 1758.61 MB
   Processing time: Complete

Dataset ready for feature extraction and drift analysis!
